# Plot Slab2 Cross Sections

This notebook can be used for creating cross sectional plots of Slab2 models. The pygmt conda environment must be installed. Grid files, trench data, clipping data, CPT files, and tilted region data are all pulled from the internet and written locally. Please note that these files can be large, and may be deleted. See web_files.py for more information.

To use this notebook, follow the instructions above cell 2 in Choosing Parameters, then run all cells.

In [1]:
import copy
import io
import numpy as np
import os
import pandas as pd
import pygmt 
import requests
import shutil
from notebook_funcs import *
from pygmt.clib import Session
from web_files import *

# Choosing Parameters

The parameters below may be changed

The user will be prompted to input 3 required values
1. The slab region
2. To specify the input date of the input data base to use as a string in the form MM-YY. The available input dates are 04-18, 12-19, 11-20, and 09-21.
3. Specify the value for n. Along with multiple, this parameter will control how the cross section location(s) is selected. When multiple is specified as True, n may either be 'all' or an integer. If n is an integer in this case, n number of equally spaced cross sectional plots will be created. If n is 'all' in this case, cross sections spaced 5 trench indices apart will be created. When multiple is specified as False, n may take the form of an integer, where n will correspond to the index of the trench file, or a list of [lon ,lat, az], corresponding to the longitude, latitude and azimuth of the cross section

The following parameters are optional, and my be edited from the default values.
* Specify, True or False, to use a color blind friendly color palette
* Specify, True or False, to plot focal mechanisms. Please note that focal mechanisms in the cross sectional view are still in development.
* Specify, True or False, to plot the input data on the overview map
* Specify, True or False, to map multiple plots based off the value of n.
* Specify the total width (in Km) of the cross section.
* The projection of 2D plots may be changed, refer to https://www.pygmt.org/latest/projections/index.html?highlight=projection for different types of available projections



In [ ]:

slab = input('Please provide the 3-letter slab code : ')
input_date = input('Please provide the input data base date in the form MM-YY. See above for options. : ')
n = input('Please provide the value or n as an int, list, or \'all\' : ')
color_blind = False
focal = False
on_map = True
multiple = False
w = 10
projection = 'M15c'

In [20]:
# Changing format of n
if n != "all" and '[' not in n:
    n = int(n)
    
elif '[' in n:
    n = n.strip('[]').split(',')
    for i in range(len(n)):
        n[i] = n[i].strip(' ')
        n[i] = float(n[i])    

# Width on either side of line is total divided by two
w /= 2

# Defining paths
path_list = paths(slab, 'xsec')
date = path_list[0]
path = path_list[1]
plot_dir = path_list[2]
xsec_dir = path_list[3]
supp_dir = path_list[4]

In [21]:
# Collecting files
file_paths = grid_links(path, date, slab)
files = file_paths[0]
clp = file_paths[1]
tilted = file_paths[2]

for file in files:
    if 'dep' in file:
        dep_file = file

# Only the depth file and clipping files are used      
file = dep_file


# Collecting relevant files for plotting
ghayes_cpt = get_ghayes(plot_dir)
bath = get_bath(plot_dir)
inputdf = get_input_df(input_date, slab)

if slab not in no_trench:
    trench = get_trench()

if color_blind == True:
    cmap = get_colorblind(plot_dir)
else:
    cmap = 'seis'

In [22]:
# Defining slabs
all_slabs = ['cal', 'cam', 'cot','hin', 'man', 'sco', 'sul', 'sam', 'cas', 'him', 
'puy', 'mak', 'hal', 'kur', 'mue', 'alu', 'ryu', 'phi', 'ker',
 'van', 'png', 'car', 'hel', 'pam', 'sol', 'sum', 'izu']
tilited_slabs = ['izu', 'ker', 'man', 'sol'] 
no_trench = ['hal', 'hin', 'pam']

In [23]:
def xsec(region: list, grd: str, z: list, az: float, lon: float, lat: float, n) -> None:
    """
    A function to plot cross sections and overview maps
    params
    ------
    region : list of [min_lon (float), max_lon (float), min_lat (float), max_lat (float)]
        the region of interest representing the min/max lon and min/max lat
    grd : str
        the path to the dep.grd file
    z : list of [min_z (float), max_z (float)]
        the min/max of the depth grid values
    az : float
        the azimuth value of the angle that is perpendicular to the trench at the cross section location
    lon : float
        longitude of the the starting point of the cross section line
    lat : float
        latitude of the the starting point of the cross section line
    n : int or list
        the number cross section currently being created
        corresponds to the row (index) of the trench file being used
        or a list containing the lat/lon/az values for a single cross section
    returns
    -------
    None
    Saves a pdf containing an overview map and plot of the cross section to a directory
    within the particular slab/date output directory
    """
    # Initial length
    length = 2000
    if slab == "izu" and 10 < lat < 15:
        length = 500

    # Axis labels
    xlabel = "Distance (Km)"
    ylabel = "Depth (Km)"
    # Plot title
    name = f"{slab} Cross Section Line {n}"
    # Reading clip file
    clip = f"{path}/{slab}_slab2_clp_{date}.csv"
    clp = pd.read_csv(clip)
    # Clipping grid
    grd = clip_grd(grd, clp, region, "dep", date, slab, projection)
    # Read trench data, if slab has trench
    if slab not in no_trench:
        df = trench
        df = df.loc[df.slab == slab]
        df = df.iloc[:, [0, 1]]

    # Projecting line perpendicular to the trench, with a starting point on trench
    # and a length of 2000
    line = pygmt.project(
        center=[lon, lat],
        azimuth=az,
        length=[-length, length],
        unit=True,
        generate="1",
    )
    # Changing lon values to degrees east
    for i in range(len(line.r)):
        if line.iloc[i, 0] < 0:
            line.iloc[i, 0] += 360
    # Constraining the line to be within the bounds of the slab
    line = pygmt.select(
        data=line.iloc[:, [0, 1]],
        F=f"{supp_dir}/poly.shp",
    )
    # Track the points on the line in the depth grd file
    try:
        track = pygmt.grdtrack(points=line, grid=grd, newcolname="tracks")
        track = track.dropna()
    except:
        print('The cross section line does not intersect the slab, please choose a different line')

    # Redefining the starting lon/lat to the beginning of the xsec line if a lon lat az was
    # given for n
    endpoint = None
    if type(n) == type([]):
        
        lon = line.r[0]
        lat = line.s[0]
        endpoint = [line.r[len(line)-1],line.s[len(line)-1]]

    # Project the track along a line perpendicular to the trench
    line2 = pygmt.project(data=track, center=[lon, lat], azimuth=az, unit=True)

    if slab in tilted_slabs:

        tilted = f"{path}/{slab}_slab2_sup_{date}.csv"  # Supplementary file
        tilted = pd.read_csv(tilted)
        tilted = tilted.iloc[:, :3]
        tilted.iloc[:, 2] = tilted.iloc[:, 2] * -1

        pw = 1  # counter for projection width increase.
        condition = None
        while condition == None and pw < 6:
            try:
                # Project tilted slab data
                line3 = pygmt.project(
                    data=tilted, center=[lon, lat], azimuth=az, unit=True, width=[0, pw]
                )

                condition = 0  # Will break the loop

                # The izu slab has length constraint issues in the first few trench indices
                # where the projection line re intersects slab. This will break the loop, as
                # interpolation is not needed in non-tilted sections, and the indices of length
                # constraint issues are outside of this region.
                if slab == "izu" and 10 < lat < 15:
                    break

                # Dropping last 10 rows of the line
                for i in range(len(line2) - 10, len(line2)):
                    line2 = line2.drop(i)
                # Filling the last rows with NaN
                empty = [np.nan for j in range(7)]
                for i in range(len(line2) - 1, len(line2) + 10):
                    line2.loc[len(line2)] = empty
                # Interpolating the last values of the normal slab to the tilted slab values
                line2 = pd.concat([line2, line3]).reset_index(drop=True)
                line2 = line2.interpolate(method="linear", limit_direction="forward")

            # Increase projection width if projection was empty
            except pd.errors.EmptyDataError:
                if pw == 1:
                    print("Issue With complex geometry")
                    print("Increasing tilted region projection width...")
                pw += 1
                if pw == 6:
                    print("Cross section not within bounds of tilted slab.")

    line2 = line2.iloc[:, [3, 2]]

    # Sometimes the lengths from the trench are negative. If so, set the zero point to the
    # first index shift all points accordingly
    if line2.iloc[0, 0] < 0:
        val = line2.iloc[0, 0] * -1
        for i in range(len(line2)):
            line2.iloc[i, 0] += val
    else:
        val = 0

    # Collecting extrema to define the axis sizes
    x1 = round((line2.iloc[:, 0]).min()) - 2
    x2 = round((line2.iloc[:, 0]).max()) + 5
    y1 = round((line2.iloc[:, 1]).min()) - 5
    x = max(abs(x2), abs(y1))
    x3 = x * -1
    if focal:
        # Square plot for focal mechanisms
        basemap_region = [-2, x, x3, 2]
        basemap_projection = "X8/8"
    else:
        basemap_region = [x1, x2, y1, z[1]]
        basemap_projection = "X16/6"
    if slab == "izu" and 10 < lat < 15:
        pass
    # else:
    length = (x2 - x1) + 20

    #################################
    # Code for XSEC Plotting Below  #
    #################################

    # Defines the input data types, and colors to be plotted with
    data_types = {
        "EQ": "skyblue",
        "AS": "grey",
        "ER": "hotpink",
        "BA": "red",
        "TO": "brown",
        "AA": "purple",
        "CP": "yellow",
        "RF": "green",
    }

    fig = pygmt.Figure()

    # Making basemap

    fig.basemap(
        region=basemap_region,
        projection=basemap_projection,
        frame=["WSne", "xaf+lDistance (Km)", "yaf+lDepth (Km)"],
    )
    # Plots the depth of the slab (surface)
    fig.plot(data=line2, pen="2p,blue", label="Slab")

    # Run through data types, collect the data for this particular xsec, and plot
    data_on_map = {}
    for key in data_types.keys():
        data = xsec_input_data(key, lon, lat, az, w, length, inputdf, n, endpoint)
        if type(data) != type(
            "string"
        ):  # input_data is set to return a str if there is no data along the xsec
            if on_map == True:
                data2 = copy.deepcopy(data)
                data_on_map[key] = data2.iloc[:, [0, 1]]
            data = data.iloc[:, [3, 2]]
            data.iloc[:, 0] += val

            if key != "BA":
                fig.plot(
                    data=data,
                    style="c.25c",
                    pen=".1,white",
                    color=data_types[key],
                    label=key,
                )
        else:
            pass

    # Plotting Focal mechanisms in the cross sectional plot below
    # DISCLAIMER - Preliminary and in development
    if focal == True:
        print(
            "DISCLAIMER -  Focal mechanism plotting within the cross sectional plot is\
 preliminary and in development"
        )
        count = 0
        focal_dict = {}
        for dat in ["EQ", "ER"]:
            # Projecting to get the correct data
            focal_data = get_focal(dat, lon, lat, az)
            focal_dict[dat] = focal_data

            if type(focal_data) != type("string"):
                for i in range(len(focal_data)):
                    if str(focal_data.iloc[i, 4]).lower() != "nan":
                        rake = float(focal_data.iloc[i, 5])
                        # Rotating rake angle to give cross sectional view
                        rake -= 90
                        focal_mechanism = dict(
                            strike=float(focal_data.iloc[i, 3]),
                            dip=float(focal_data.iloc[i, 4]),
                            rake=rake,
                            magnitude=float(focal_data.iloc[i, 6]),
                        )
                        fig.meca(
                            focal_mechanism,
                            scale=".25c",
                            longitude=float(focal_data.iloc[i, 7]),
                            latitude=float(focal_data.iloc[i, 2]),
                            depth=float(focal_data.iloc[i, 2]),
                        )

                        count += 1

                    else:
                        pass
                if count == 0:
                    print(f"No Focal Mechanism data for {dat} in this line")
                else:
                    pass
            else:
                pass

    fig.legend(box=True, position="JBL+jBL+o0.2c", region=basemap_region)

    fig.shift_origin(yshift=1)

    fig.text(text="A`", position="TR", font="28p,Helvetica-Bold,black")

    fig.text(text="A", position="Tl", font="28p,Helvetica-Bold,black")
    # Plotting Overview Map Below
    fig.grdimage(
        grid=bath,
        nan_transparent=True,
        region=region,
        projection=projection,
        cmap=ghayes_cpt,
        yshift=+11,
    )

    fig.coast(region=region, projection=projection, frame=["ag"], shorelines=True)

    if slab not in tilted_slabs:
        fig.grdimage(
            grid=grd,
            nan_transparent=True,
            region=region,
            transparency=20,
            projection=projection,
            cmap="cpt.cpt",
        )
    else:

        tilted = f"{path}/{slab}_slab2_sup_{date}.csv"  # Supplementary file
        tilted = pd.read_csv(tilted)
        tilted = tilted.iloc[:, :3]

        fig.grdimage(
            grid=grd,
            nan_transparent=True,
            region=region,
            transparency=20,
            projection=projection,
            cmap="cpt.cpt",
        )

        cpt = pygmt.makecpt(
            cmap=cmap,
            series=f"{str(-1*z[1])}/{str(-1*z[0])}/10",
            background=True,
            color_model="r",
            continuous=True,
            output="cpt2.cpt",
        )

        # Plotting complex dep surface
        fig.plot(
            data=tilted,
            projection=projection,
            region=region,
            style="c0.08",
            cmap="cpt2.cpt",
        )
    # Plotting cross section line on map
    fig.plot(
        x=line.r,
        y=line.s,
        pen="2p,blue",
        region=region,
        projection=projection,
        frame=["ag", f"+t{name}"],
        label="Cross section",
    )
    # Adding markers on map if specified
    if on_map == True:
        for key in data_on_map.keys():
            data2 = data_on_map[key]
            fig.plot(
                data=data2,
                region=region,
                projection=projection,
                style="c.1c",
                pen=".1,black",
                color=data_types[key],
                label=key,
            )
    # Plotting Focal Mechanisms on map
    if focal == True:
        for dat in ["EQ", "ER"]:
            focal_data = focal_dict[dat]
            if type(focal_data) != type("string"):
                for i in range(len(focal_data)):
                    if str(focal_data.iloc[i, 4]) != "nan":
                        focal_mechanism = dict(
                            strike=float(focal_data.iloc[i, 3]),
                            dip=float(focal_data.iloc[i, 4]),
                            rake=float(focal_data.iloc[i, 5]),
                            magnitude=float(focal_data.iloc[i, 6]),
                        )
                        fig.meca(
                            focal_mechanism,
                            scale=".25c",
                            longitude=float(focal_data.iloc[i, 0]),
                            latitude=float(focal_data.iloc[i, 1]),
                            depth=float(focal_data.iloc[i, 2]),
                        )
                    else:
                        pass
            else:
                pass
    # Plotting clip to add outline
    fig.plot(data=clp, pen=".5,black", region=region, projection=projection)
    if slab not in no_trench:
        # Plotting the trench as a yellow line
        fig.plot(data=df, projection=projection, pen="2,yellow", label="Trench")
    # Adding contours
    fig.grdcontour(grid=grd, projection=projection)
    fig.colorbar(cmap="cpt.cpt", frame=[f"x+lDepth(Km)"])
    fig.legend(box=True)

    fig.text(
        x=[line.r[0], line.r[len(line) - 1]],
        y=[line.s[0], line.s[len(line) - 1]],
        text=["A", "A`"],
        font="10p,Helvetica-Bold,white",
        fill="black",
        transparency=10,
    )
    print(f"Saving plot in {xsec_dir}/{slab}_xsec_{n}.pdf")
    fig.savefig(f"{xsec_dir}/{slab}_xsec_{n}.pdf")
    if n != 'all':
        fig.show()

In [24]:
def main(n: int or str, file: str) -> None:
    info = get_info(file, slab)
    region = info[0]
    z = info[1]
    # Creating depth CPT
    make_cpt(cmap,z)
    df = trench
    df = df.loc[df.slab == slab]
    df = df.iloc[:,[0,1,2]]
    
    if n == 'all':
        for i in range(0,len(df),5):
            
            info = index_info(i,df)
            lon = info[0]
            lat = info[1]
            az = info[2]
            try:
                xsec(region, file, z, az, lon, lat, i)
            except:
                print('Could not make an XSEC, skipping this line.')
                pass
        exit()
    
    # Looping through trench rows
    count = 1
    adder = round(len(df)/n)
    i = 0
    while count <= n:
        try:
            info = index_info(i,df)
            lon = info[0]
            lat = info[1]
            az = info[2]
            condition = None
            while condition is None:
                try:
                    xsec(region, file, z, az, lon, lat, i)
                except:
                    print('Could not make an XSEC, skipping this line.')
                    i += 1
                    info = index_info(i,df)
                    lon = info[0]
                    lat = info[1]
                    az = info[2]
                    pass
            
                else:
                    i += adder
                    count += 1
                    condition = 0
        except IndexError:
            i = len(df)-1
            info = index_info(i,df)
            lon = info[0]
            lat = info[1]
            az = info[2]
            count += n 
            condition = None
            while condition is None:
                try:
                    xsec(region, file, z, az, lon, lat, i)
                except:
                    print('Could not make an XSEC, skipping this line.')
                    i -= 1
                    info = index_info(i,df)
                    lon = info[0]
                    lat = info[1]
                    az = info[2]
                    pass
            
                else:
                    condition = 0

In [25]:
def single(n,file):
    """
    A function to create a single cross section
    The same function as main, but does not loop through trench rows
    
    Parameters
    ------
    n : int or list
         if int : the particular row of the trench file to create a cross section for
         if list : the [lon, lat, az] to make cross section at

    Returns
    -------
    None
    """
    info = get_info(file, slab)
    region = info[0]
    z = info[1]
    make_cpt(cmap,z)
    if type(n) == type(1):
        try:
            df = trench
            df = df.loc[df.slab == slab]
            df = df.iloc[:,[0,1,2]]
            info = index_info(n,df)
            lon = info[0]
            lat = info[1]
            az = info[2]
            xsec(region, file, z, az, lon, lat, n)
            
        except KeyError:# IndexError:
            print('The value you chose for the cross section number is out of range')
            print(f'Please pick a value from 0-{len(df)}')
    else:
        lon = float(n[0])
        lat = float(n[1])
        az = float(n[2])
        if -180 <= lon <= 360 and -180 <= lat <= 180 and 0 <= az <=360:
            n = [lon,lat,az]
            info = get_info(file, slab)
            region = info[0]
            z = info[1]
            xsec(region, file, z, az, lon, lat, n)
        else:
            print('Please input valid values for [lon,lat,az]')
            quit()
    
    


In [ ]:
if __name__ == "__main__":
    
    if multiple or  n == 'all':
        main(n,file)
    else:
        single(n,file)